In [31]:
import os
import shutil
import unicodedata
import copy

import pandas as pd
from tqdm import tqdm

In [3]:
fromfd = "/home/jupyter/ai_font/data/exp0604/raw/카엔플린"
tofd = "/home/jupyter/ai_font/data/exp0604/test/pngs"

In [6]:
files = [f for f in os.listdir(fromfd) if f.endswith(".png")]
for f in files:
    frompath = f"{fromfd}/{f}"
    f = unicodedata.normalize("NFC", f)
    topath = f"{tofd}/카엔플린__{f}"
    shutil.copy(frompath, topath)

In [7]:
# Create testmap

In [10]:
def get_lexicon(letter):
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    # lengths = [19,21,28]
    return ch1, ch2, ch3

def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns


In [11]:
ak = get_all_korean()

In [13]:
listdf = []
for k in ak:
    c1, c2, c3 = get_lexicon(k)
    listdf.append({
        "letter": k,
        "초성": c1,
        "중성": c2,
        "종성": c3,
    })

In [17]:
akdf = pd.DataFrame().from_dict(listdf)

In [18]:
akdf.index = akdf.letter

In [19]:
akdf

,letter,초성,중성,종성
letter,,,,
가,가,0,0,0
각,각,0,0,1
갂,갂,0,0,2
갃,갃,0,0,3
간,간,0,0,4
...,...,...,...,...
힟,힟,18,20,23
힠,힠,18,20,24
힡,힡,18,20,25


### 유사한 글자 찾기 우선순위

1. 초성과 중성이 같은 글자
2. 중성과 종성이 같은 글자
3. 초성과 종성이 같은 글자

1~3을 교차로 배치한다

4. 초성 같은 글자
5. 중성 같은 글자
6. 종성 같은 글자

4~6을 교차로 배치한다


In [35]:
fd = "/home/jupyter/ai_font/data/exp0604/test/pngs"
for d in tqdm(listdf):
    similar = list()
    l1 = akdf.loc[(akdf['초성']==d['초성'])&(akdf['중성']==d['중성'])].letter.values
    l2 = akdf.loc[(akdf['중성']==d['중성'])&(akdf['종성']==d['종성'])].letter.values
    l3 = akdf.loc[(akdf['초성']==d['초성'])&(akdf['종성']==d['종성'])].letter.values
    iterlen = max([len(l1),len(l2),len(l3)])
    for i in range(iterlen):
        if i < len(l1):
            path1 = f"{fd}/카엔플린__{l1[i]}.png"
            if os.path.exists(path1) & (l1[i]!=d['letter']):
                similar.append(l1[i])
                # print(l1[i])
        if i < len(l2):
            path2 = f"{fd}/카엔플린__{l2[i]}.png"
            if os.path.exists(path2) & (l2[i]!=d['letter']):
                similar.append(l2[i])
                # print(l2[i])
        if i < len(l3):
            path3 = f"{fd}/카엔플린__{l3[i]}.png"
            if os.path.exists(path3) & (l3[i]!=d['letter']):
                similar.append(l3[i])
                # print(l3[i])
    d["similar"] = copy.deepcopy(similar)
    
for d in tqdm(listdf):
    similar = list()
    l1 = akdf.loc[(akdf['초성']==d['초성'])].letter.values
    l2 = akdf.loc[(akdf['중성']==d['중성'])].letter.values
    l3 = akdf.loc[(akdf['종성']==d['종성'])].letter.values
    iterlen = max([len(l1),len(l2),len(l3)])
    for i in range(iterlen):
        if i < len(l1):
            path1 = f"{fd}/카엔플린__{l1[i]}.png"
            if os.path.exists(path1) & (l1[i]!=d['letter']):
                similar.append(l1[i])
                # print(l1[i])
        if i < len(l2):
            path2 = f"{fd}/카엔플린__{l2[i]}.png"
            if os.path.exists(path2) & (l2[i]!=d['letter']):
                similar.append(l2[i])
                # print(l2[i])
        if i < len(l3):
            path3 = f"{fd}/카엔플린__{l3[i]}.png"
            if os.path.exists(path3) & (l3[i]!=d['letter']):
                similar.append(l3[i])
                # print(l3[i])
    d["similar"] += copy.deepcopy(similar)
    


100%|██████████| 11172/11172 [01:01<00:00, 180.40it/s]


In [37]:
pd.DataFrame().from_dict(listdf).to_pickle("/home/jupyter/ai_font/data/exp0604/pickle/카엔플린_testmap.pickle")